# ICMP Pinger

Ping is a computer network application used to test whether a particular host is reachable across an IP
network. It is also used to self-test the network interface card of the computer or as a latency test. It works
by sending ICMP “echo reply” packets to the target host and listening for ICMP “echo reply” replies. The
"echo reply" is sometimes called a pong. Ping measures the round-trip time, records packet loss, and
prints a statistical summary of the echo reply packets received (the minimum, maximum, and the mean of
the round-trip times and in some versions the standard deviation of the mean).



Your task is to develop your own Ping application in Python. Your application will use ICMP but, in
order to keep it simple, will not exactly follow the official specification in RFC 1739. Note that you will
only need to write the client side of the program, as the functionality needed on the server side is built
into almost all operating systems.



You should complete the Ping application so that it sends ping requests to a specified host separated by
approximately one second. Each message contains a payload of data that includes a timestamp. After
sending each packet, the application waits up to one second to receive a reply. If one second goes by
without a reply from the server, then the client assumes that either the ping packet or the pong packet was
lost in the network (or that the server is down).

In [ ]:
from socket import *
import os
import sys
import struct
import time
import select
import binascii

ICMP_ECHO_REQUEST = 8 #defines the ICMP type for an Echo Request message(type 8)


def checksum(source_string):
    csum = 0
    countTo = (len(source_string) // 2) * 2
    count = 0

    while count < countTo:
        thisVal = int.from_bytes(source_string[count:count+2], "big")
        csum = csum + thisVal
        csum = csum & 0xffffffff
        count = count + 2

    if countTo < len(source_string):
        csum = csum + source_string[-1]
        csum = csum & 0xffffffff

    csum = (csum >> 16) + (csum & 0xffff)
    csum = csum + (csum >> 16)
    answer = ~csum & 0xffff
    answer = answer >> 8 | (answer << 8 & 0xff00)
    return answer



def receiveOnePing(mySocket, ID, timeout, destAddr): # receiving pong from the target host after sending a ping request.
    timeLeft = timeout

    while 1:
        startedSelect = time.time()
        whatReady = select.select([mySocket], [], [], timeLeft)
        howLongInSelect = (time.time() - startedSelect)

        if whatReady[0] == []:  # Timeout
            return "Request timed out."

        timeReceived = time.time()
        recPacket, addr = mySocket.recvfrom(1024)

        # Fill in start
        # Fetch the ICMP header from the IP packet
        icmpHeader = recPacket[20:28] #ICMP header (20-28 bytes) is extracted from the received packet.
        type, code, checksum, id, sequence = struct.unpack("bbHHh", icmpHeader) # extracts type (8), code (8), checksum (16), id (16), sequence (16)
        
        if id == ID:
            return (timeReceived - struct.unpack("d", recPacket[28:])[0]) * 1000 # converts time into miliseconds
        # Fill in end
        timeLeft = timeLeft - howLongInSelect
        if timeLeft <= 0:
            return "Request timed out."


def sendOnePing(mySocket, destAddr, ID): # constructs and sends an ICMP Echo Request (ping) packet
    # Header is type (8), code (8), checksum (16), id (16), sequence (16)
    myChecksum = 0
    # Make a dummy header with a 0 checksum
    # struct -- Interpret strings as packed binary data
    header = struct.pack("bbHHh", ICMP_ECHO_REQUEST, 0, myChecksum, ID, 1)
    data = struct.pack("d", time.time())
    # Calculate the checksum on the data and the dummy header.
    myChecksum = checksum(header + data)
    # Get the right checksum, and put in the header
    if sys.platform == 'darwin':
        # Convert 16-bit integers from host to network byte order
        myChecksum = htons(myChecksum) & 0xffff
    else:
        myChecksum = htons(myChecksum)

    header = struct.pack("bbHHh", ICMP_ECHO_REQUEST, 0, myChecksum, ID, 1)
    packet = header + data
    mySocket.sendto(packet, (destAddr, 1))  # AF_INET address must be tuple, not str


def doOnePing(destAddr, timeout):
    icmp = getprotobyname("icmp")
    # SOCK_RAW is a powerful socket type.
    # For more details: http://sockraw.org/papers/sock_raw
    mySocket = socket(AF_INET, SOCK_RAW, icmp)
    myID = os.getpid() & 0xFFFF  # Return the current process i
    sendOnePing(mySocket, destAddr, myID)
    delay = receiveOnePing(mySocket, myID, timeout, destAddr)
    mySocket.close()
    return delay


def ping(host, timeout=1):
    # timeout=1 means: If one second goes by without a reply from the server,
    # the client assumes that either the client's ping or the server's pong is lost
    dest = gethostbyname(host)
    print("Pinging " + dest + " using Python:")
    print("")
    # Send ping requests to a server separated by approximately one second
    while 1:
        delay = doOnePing(dest, timeout)
        print(delay)
        time.sleep(1)  # one second
    return delay


ping("127.0.0.1")


First, test your client by sending packets to localhost, that is, 127.0.0.1.
Then, you should see how your Pinger application communicates across the network by pinging servers
in different continents.

checksum: calculates the Internet checksum for an ICMP packet.

1) Initialize csum to 0
2) Data is split into 16-bit words
3) Sum all the 16-bit words
4) If the sum is greater than 16 bits, the extra bits are wrapped around and added back.
5) Take the one's complement
6) The final step is to flip all bits to produce the checksum.
7) The receiver performs the same checksum calculation on the received data.
If the result is 0xFFFF, the packet is considered intact.
If not, it means there was an error in transmission.


receiveOnePing(): responsible for receiving the pong from the target host after sending a ping request. 

1) timeLeft = timeout: setting the remaining wait time for a reply.
2) select.select([mySocket], [], [], timeLeft): waits for the socket to receive data. If nothing is received within the timeout period, it returns "Request timed out."
3) mySocket.recvfrom(1024): it captures the packet and the sender's address.
4) icmpHeader = recPacket[20:28]: ICMP header (20-28 bytes) is extracted from the received packet.
5) struct.unpack("bbHHh", icmpHeader): extracts type (8), code (8), checksum (16), id (16), sequence (16)

sendOnePing: constructs and sends an ICMP Echo Request (ping) packet

doOnePing: Creates a raw socket for ICMP then sends a ping request with a unique identifier and waits for a response and measures round-trip time
